In [0]:
# =============================================================================
# 10. SESSÃO DE MONITORAMENTO E ANÁLISE PÓS-EXECUÇÃO
# =============================================================================
print("\n=============================================================================")
print("INFO: Iniciando sessão de monitoramento e análise dos dados gerados...")
print("=============================================================================")

from pyspark.sql import functions as F

# Carregar as tabelas recém-criadas
try:
    df_clientes_final = spark.table("transacoes_db.copper.clientes")
    df_contas_final = spark.table("transacoes_db.copper.contas")
    df_chaves_final = spark.table("transacoes_db.copper.chaves_pix")
    df_transacoes_final = spark.table("transacoes_db.copper.transacoes")
    print("INFO: Tabelas Delta carregadas com sucesso para análise.")
except Exception as e:
    print(f"ERRO: Não foi possível carregar as tabelas para análise. Verifique se foram criadas corretamente. Erro: {e}")
    # Se não conseguir carregar as tabelas, encerra a análise.
    # Usando 'raise' para interromper o script se as tabelas forem essenciais.
    raise e

# --- Análise 1: Perfil Geral da População ---
print("\n----- Análise 1: Perfil da População -----")
total_clientes = df_clientes_final.count()
total_contas = df_contas_final.count()
total_chaves = df_chaves_final.count()

print(f"  - Total de Clientes Gerados: {total_clientes}")
print(f"  - Total de Contas Geradas: {total_contas}")
print(f"  - Total de Chaves PIX Geradas: {total_chaves}")

# Detalhamento de Clientes PF vs PJ
df_clientes_natureza = df_clientes_final.groupBy("id_natureza").count().withColumn(
    "tipo_cliente",
    F.when(F.col("id_natureza") == 1, "Pessoa Física")
    .when(F.col("id_natureza") == 2, "Pessoa Jurídica")
    .otherwise("Não Definido")
)
print("\nDistribuição de Clientes por Natureza:")
# Em notebooks (Databricks/Jupyter), a função display() gera gráficos automaticamente
# Se estiver rodando como script .py, use .show()
try:
    display(df_clientes_natureza.select("tipo_cliente", "count"))
except NameError:
    df_clientes_natureza.select("tipo_cliente", "count").show()


# --- Análise 2: Perfil das Transações ---
print("\n----- Análise 2: Perfil das Transações -----")
total_transacoes = df_transacoes_final.count()
valor_total_transacionado = df_transacoes_final.agg(F.sum("valor")).first()[0]

print(f"  - Total de Transações Geradas (incluindo dispersão de fraude): {total_transacoes}")
print(f"  - Valor Total Transacionado: R$ {valor_total_transacionado:,.2f}")

# Volume de Transações por Mês
df_transacoes_por_mes = (
    df_transacoes_final
    .withColumn("mes", F.month(F.col("data")))
    .groupBy("mes")
    .count()
    .orderBy("mes")
)
print("\nVolume de Transações por Mês:")
try:
    display(df_transacoes_por_mes)
except NameError:
    df_transacoes_por_mes.show()


# --- Análise 3: Análise de Fraudes ---
print("\n----- Análise 3: Análise de Fraudes -----")
df_fraudes = df_transacoes_final.filter(F.col("is_fraud") == 1)
df_legitimas = df_transacoes_final.filter(F.col("is_fraud") == 0)

total_fraudes = df_fraudes.count()
valor_total_fraudado = df_fraudes.agg(F.sum("valor")).first()[0]
percentual_fraude_volume = (total_fraudes / total_transacoes) * 100 if total_transacoes > 0 else 0
percentual_fraude_valor = (valor_total_fraudado / valor_total_transacionado) * 100 if valor_total_transacionado > 0 else 0

print(f"  - Volume de Transações Fraudulentas: {total_fraudes} ({percentual_fraude_volume:.2f}%)")
print(f"  - Valor Total Fraudado: R$ {valor_total_fraudado:,.2f} ({percentual_fraude_valor:.2f}%)")

# Média de valor por tipo de transação
media_valor_fraude = df_fraudes.agg(F.avg("valor")).first()[0]
media_valor_legitima = df_legitimas.agg(F.avg("valor")).first()[0]
print(f"  - Valor Médio da Fraude: R$ {media_valor_fraude:,.2f}")
print(f"  - Valor Médio da Transação Legítima: R$ {media_valor_legitima:,.2f}")

# Distribuição dos Tipos de Fraude
df_tipos_fraude = df_fraudes.groupBy("fraud_type").count()
print("\nDistribuição dos Tipos de Fraude:")
try:
    display(df_tipos_fraude)
except NameError:
    df_tipos_fraude.show()

# Análise da Cadeia de Fraude (Raiz vs. Dispersão)
transacoes_raiz_fraude = df_fraudes.filter(F.col("id_transacao_cadeia_pai").isNull()).count()
transacoes_dispersao_fraude = df_fraudes.filter(F.col("id_transacao_cadeia_pai").isNotNull()).count()
print("\nEstrutura da Cadeia de Fraude:")
print(f"  - Transações Raiz (Início da Fraude): {transacoes_raiz_fraude}")
print(f"  - Transações de Dispersão (Lavagem): {transacoes_dispersao_fraude}")


# --- Análise 4: Contas de Alto Risco (High Risk) ---
print("\n----- Análise 4: Contas de Alto Risco -----")
contas_alto_risco = df_contas_final.filter(F.col("is_high_risk") == 1).count()
percentual_alto_risco = (contas_alto_risco / total_contas) * 100 if total_contas > 0 else 0
print(f"  - Total de Contas Marcadas como Alto Risco: {contas_alto_risco} ({percentual_alto_risco:.2f}%)")

# Verifica o envolvimento de contas de alto risco como destino de transações
df_destino_risco = (
    df_transacoes_final
    .join(df_contas_final.alias("contas_destino"), F.col("id_conta_destino") == F.col("contas_destino.id"), "inner")
    .filter(F.col("contas_destino.is_high_risk") == 1)
)

tx_para_conta_risco = df_destino_risco.count()
tx_fraude_para_conta_risco = df_destino_risco.filter(F.col("is_fraud") == 1).count()
percentual_envolvimento = (tx_fraude_para_conta_risco / total_fraudes) * 100 if total_fraudes > 0 else 0

print(f"  - Total de transações destinadas a contas de alto risco: {tx_para_conta_risco}")
print(f"  - Fraudes destinadas a contas de alto risco: {tx_fraude_para_conta_risco} ({percentual_envolvimento:.2f}% de todas as fraudes)")

print("\n=============================================================================")
print("INFO: Sessão de monitoramento e análise CONCLUÍDA.")
print("=============================================================================")